In [10]:
import pandas as pd

In [11]:
data = pd.read_csv('cleaned_data_.csv')

In [12]:
def convert_to_days(time_str):
    try:
        # Extract numeric values from the string
        times = [float(t) for t in time_str.split() if t.replace('.', '').isdigit()]
        
        # Convert to days based on the presence of "week" or "month"
        if "week" in time_str:
            times = [t * 7 for t in times]
        elif "month" in time_str:
            times = [t * 30 for t in times]  # Assuming 30 days per month
        
        # Return average if multiple values are present, else return the single value
        return sum(times) / len(times) if times else None
    except Exception as e:
        return None


# Preprocessing

### Creating Input Output Sentences for BERT

In [13]:
# Extracting relevant columns related to expiration time for different locations
relevant_columns_with_temp = [
    ("Pantry_Text", "pantry", 60),
    ("Refrigerate_Text", "refrigerator", 40),
    ("DateOfPurchase_Freeze_Text", "freezer", 0),
    # Add more columns as per requirements
]

# Initialize lists to hold the input and output sentences
input_sentences_with_temp = []
output_days_with_temp = []

# Loop through the data
for _, row in data.iterrows():
    for col, location, temp in relevant_columns_with_temp:
        # Check if expiration data is available
        if pd.notna(row[col]):
            # Construct the input sentence with temperature
            input_sentence = f"I am storing {row['Name']} in my {location} at {temp}°F, how long until it spoils or goes bad?"
            # Convert the output to days
            output_day = convert_to_days(row[col])
            
            # Append to respective lists if output_day is not None
            if output_day is not None:
                input_sentences_with_temp.append(input_sentence)
                output_days_with_temp.append(output_day)

# Display first few input-output pairs with temperature consideration
sample_data_days_with_temp = pd.DataFrame({
    "Input": input_sentences_with_temp[:5],
    "Output (Days)": output_days_with_temp[:5]
})
sample_data_days_with_temp


,Input,Output (Days)
0,I am storing fruit cocktail in my pantry at 60...,540.0


### Tokenizing

In [16]:
from transformers import DistilBertConfig, DistilBertModel
import torch

RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
libssl.so.10: cannot open shared object file: No such file or directory